In [1]:
import pandas as pd
import numpy as np
import pyod
import matplotlib.pyplot as pyplot
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn import preprocessing
from pyod.utils.data import evaluate_print

from pyod.models.xgbod import XGBOD

from datetime import datetime

np.set_printoptions(precision=3)

In [2]:
testing_set = pd.read_csv('../Balanced_Samples/Sample_1K.csv',sep=',',header=0)
training_set = pd.read_csv('../Balanced_Samples/Sample_20K.csv',sep=',',header=0)

C:\Users\andre\Anaconda3\envs\security\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#transforma dados categóricos em números
for f in testing_set.columns: 
    if testing_set[f].dtype=='object': 
        label = preprocessing.LabelEncoder() 
        label.fit(list(testing_set[f].values)) 
        testing_set[f] = label.transform(list(testing_set[f].values))
    
testingSet = testing_set.values
np.random.shuffle(testingSet)

testingSet = testingSet.astype(float)

for i in range (1*10**3-1, 0, -1):
    for j in range(0, 84):
        testingSet[i, j] = float(testingSet[i, j])
        if (np.isinf(testingSet[i, j]) or np.isnan(testingSet[i, j])):
            testingSet = np.delete(testingSet, i, axis=0)

y_test = testingSet[:, 84].astype(int)

In [4]:
#transforma dados categóricos em números
for f in training_set.columns: 
    if training_set[f].dtype=='object': 
        label = preprocessing.LabelEncoder() 
        label.fit(list(training_set[f].values)) 
        training_set[f] = label.transform(list(training_set[f].values))
    
trainingSet = training_set.values
np.random.shuffle(trainingSet)

trainingSet = trainingSet.astype(float)

for i in range (2*10**4-1, 0, -1):
    for j in range(0, 84):
        trainingSet[i, j] = float(trainingSet[i, j])
        if (np.isinf(trainingSet[i, j]) or np.isnan(trainingSet[i, j])):
            trainingSet = np.delete(trainingSet, i, axis=0)

y_train = trainingSet[:, 84].astype(int)

In [5]:
outliers_fraction = 0.5
def testMethod(clf, clf_name):
    print("Started fitting: ", datetime.now(), "\n")
    clf.fit(trainingSet, y_train)
    print("Endend fitting: ", datetime.now(), "\n")

    #####################################################################
    
    print("Started predicting: ", datetime.now(), "\n")
    y_test_pred = clf.predict(testingSet)  # outlier labels (0 or 1)
    y_test_scores = clf.decision_function(testingSet)  # outlier scores
    print("Ended predicting: ", datetime.now(), "\n")
    
    #####################################################################

    print("\nOn Test Data - "+clf_name+":")
    truePositive = 0
    trueNegative = 0
    falsePositive = 0
    falseNegative = 0

    for i in range(y_test.size):
        if(y_test[i] == 1 and y_test_pred[i] == 1):
            truePositive = truePositive+1
        elif (y_test[i] == 0 and y_test_pred[i] == 0):
            trueNegative = trueNegative+1
        elif (y_test[i] == 0 and y_test_pred[i] == 1):
            falsePositive = falsePositive+1
        else:
            falseNegative = falseNegative+1

    print(truePositive, " | ", falsePositive)
    print(falseNegative, " | ", trueNegative,"\n")

    evaluate_print(clf_name, y_test, y_test_scores)

    #####################################################################

    fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_scores)
    roc_auc = metrics.auc(fpr, tpr)

    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic - '+clf_name)
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
testMethod(XGBOD(contamination= outliers_fraction), "XGBOD")

Started fitting:  2019-10-13 16:32:26.108439 

Fitting estimator 1 of 68 (standarlization: true):
 KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=1, p=2,
  radius=1.0)
Start:  2019-10-13 16:32:26.250350


C:\Users\andre\Anaconda3\envs\security\lib\site-packages\pyod\models\base.py:344: UserWarning: y should not be presented in unsupervised learning.
  "y should not be presented in unsupervised learning.")


End:  2019-10-13 16:32:41.114190 

Fitting estimator 2 of 68 (standarlization: true):
 KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='mean',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=1, p=2,
  radius=1.0)
Start:  2019-10-13 16:32:41.125183
End:  2019-10-13 16:32:58.102722 

Fitting estimator 3 of 68 (standarlization: true):
 LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=1, p=2)
Start:  2019-10-13 16:32:58.105720
End:  2019-10-13 16:33:13.369317 

Fitting estimator 4 of 68 (standarlization: true):
 KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=3, p=2,
  radius=1.0)
Start:  2019-10-13 16:33:13.372316
End:  2019-10-13 16:33:33.534889 

Fitting estimator 5 of 68 (standarlization: true):
 KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='mean',
  metric='minkowski', metric_params